In [1]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

from dcopf_utils import build_gurobi_model#, build_osqp_model
import time
from dcopf_utils import DcopfProblem
from utils import build_ipopt

In [2]:
from model_utils import NNSolver_eq_proj

filepath = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf50_data"
with open(filepath, 'rb') as f:
    dataset = pickle.load(f)
data = DcopfProblem(dataset, valid_frac=0.0833, test_frac=0.001)


arg_path = "/home/jxxiong/A-xjx/DC3/results/DcopfProblem-50-25-10-1200/method_eq_proj/fdb937677208d3fd9415c7c92b2f14cfc38dc5d7/1714062081-314273/args.dict"
args = pickle.load(open(arg_path, 'rb'))

model = NNSolver_eq_proj(data, args)
model.to('cpu')
model_path = "/home/jxxiong/A-xjx/DC3/results/DcopfProblem-50-25-10-1200/method_eq_proj/fdb937677208d3fd9415c7c92b2f14cfc38dc5d7/1714062081-314273/solver_net.dict"
model.load_state_dict(torch.load(model_path))

model.eval()

NNSolver_eq_proj(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=50, bias=True)
  )
)

In [3]:
DEVICE = 'cpu'
model.to(DEVICE)
for attr in dir(data):
    var = getattr(data, attr)
    if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
        try:
            setattr(data, attr, var.to(DEVICE))
        except AttributeError:
            pass

In [4]:
A = data.A
G = data.G
h = data.h
Q = data.Q
p = data.p
Lb = data.Lb
Ub = data.Ub

A_np = data.A_np
G_np = data.G_np
h_np = data.h_np
Q_np = data.Q_np
p_np = data.p_np
Lb_np = data.Lb.cpu().numpy()
Ub_np = data.Ub.cpu().numpy()

X = data.X
X_np = data.X_np

num_var = A.shape[1]
num_eq = A.shape[0]
num_ineq = G.shape[0]

In [5]:
p.max()

tensor(3.9554)

In [6]:
X.shape

torch.Size([1200, 10])

In [344]:
idx = 1164

In [345]:
# full_model = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
# start_time = time.time()
# r = full_model.solve()
# print("OSQP time: ", time.time() - start_time)
# print(r.info.obj_val)
# x_full = r.x

In [346]:
# Lb_np_reduced = Lb_np.copy()
# Lb_np_reduced[inactive_lower_bounds] = -np.inf
# Ub_np_reduced = Ub_np.copy()
# Ub_np_reduced[inactive_upper_bounds] = np.inf

# reduced_model = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np_reduced, Ub_np_reduced)
# start_time = time.time()
# r = reduced_model.solve()
# print("OSQP reduced time: ", time.time() - start_time)
# print(r.info.obj_val)
# x_reduced = r.x

In [347]:
# def get_init(A, x):
#     y0 = np.linalg.pinv(A)@x
#     return y0

# full_model = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
# start_time = time.time()
# optimal_x_full, info_full = full_model.solve(get_init(A_np, X_np[idx]))
# end_time = time.time()
# time_full = end_time - start_time
# print(info_full['obj_val'], time_full)

In [348]:
model_presolve = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
%time model_presolve = model_presolve.presolve()
# reduced_model.write("reduced_model.lp")

CPU times: user 4.14 ms, sys: 0 ns, total: 4.14 ms
Wall time: 10.2 ms


In [349]:
full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
full_model.setParam('OutputFlag', 1)
full_model.setParam('Presolve', 1)
full_model.optimize()
optimal_val_full = full_model.objVal
time_full = full_model.Runtime
optimal_x_full = np.array([v.x for v in full_model.getVars()])
# duals_full = np.array([c.Pi for c in full_model.getConstrs()])
# duals_ineq_target = duals_full[-G.shape[0]:]
# duals_eq_target = duals_full[:A.shape[0]]
active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
active_ineq_target.tolist()

active_lower_bounds = np.where(optimal_x_full - Lb_np <= 1e-4)[0]
active_upper_bounds = np.where(Ub_np - optimal_x_full <= 1e-4)[0]
print(active_ineq_target)
print(active_lower_bounds)
print(active_upper_bounds)

Set parameter OutputFlag to value 1
Set parameter Presolve to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 35 rows, 50 columns and 977 nonzeros
Model fingerprint: 0xe86d59bf
Model has 1213 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-03, 4e+00]
  Objective range  [2e-01, 4e+00]
  QObjective range [2e-04, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 2376 rows, 1264 columns, 6871 nonzeros
Presolved model has 36 quadratic constraint(s)
Presolved model has 1177 bilinear constraint(s)
Variable types: 1264 continuous, 0 integer (0 binary)
Found heuristic solution: objective 7.6119300

Root relaxation: objective -1.086365e+00, 

In [333]:
full_model.objVal

4.293029214395894

In [334]:
inactive_lower_bounds = np.where(optimal_x_full - Lb_np > 1e-4)[0]
inactive_upper_bounds = np.where(Ub_np - optimal_x_full > 1e-4)[0]

In [335]:
# calculate the parameter values
# num_ineq = 11424 + 7024*2
opt_gap = 1e-2
# fac = max(1, max(np.abs(Q_np).max(), np.abs(p_np).max()))
fac = 1
t = min(fac * (num_ineq + num_var * 2) / opt_gap / abs(optimal_val_full), 1000)
dual_threshold = 1e-3
# ineq_threshold = 1 / dual_threshold / (int(t))
ineq_threshold = 1.0

print(int(t), ineq_threshold)

1000 1.0


In [336]:
(t / 5) ** (1/30)

1.1931663584704937

# remove part of the inequality constraints

In [337]:
# start_time = time.time()
# Yhat = model(X[idx].unsqueeze(0).to('cuda'))
# Ystar = data.projection(X[idx].unsqueeze(0).to('cuda'), Yhat)
# end_time = time.time()


In [338]:
start_time = time.time()
Yhat = model(X[idx].unsqueeze(0))
Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
end_time = time.time()
inference_time = end_time - start_time
print("Inference time: ", inference_time)
Ystar = Ystar.detach().numpy().flatten()
ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
ineq_sorted = np.sort(-G_np@Ystar + h_np)
print(ineq_sorted)
print(ineq_sorted_index)
inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]
print("inactive ineq: ", inactive_ineq_index)

lb_sorted_index = np.argsort(Ystar - Lb_np)
lb_sorted = np.sort(Ystar - Lb_np)
inactive_lower_bounds = lb_sorted_index[lb_sorted > ineq_threshold]

ub_sorted_index = np.argsort(Ub_np - Ystar)
ub_sorted = np.sort(Ub_np - Ystar)
inactive_upper_bounds = ub_sorted_index[ub_sorted > ineq_threshold]
print(inactive_lower_bounds)
print(inactive_upper_bounds)


Inference time:  0.0014226436614990234
[0.03379015 0.04693262 0.07258627 0.47910076 1.62363172 1.80027958
 1.88999732 1.95820351 1.99381102 2.79169598 2.96021677 3.15057323
 3.49037426 3.55824665 3.81515021 4.09859577 4.16647625 4.27583579
 5.08362688 5.52158768 5.53596778 6.06743764 6.5014343  6.65214374
 7.37438974]
[ 0 21 16  8 11 24 18 19 22  4 23 10 14 20  2 13  7 12 17  5  1  6  3 15
  9]
inactive ineq:  [11 24 18 19 22  4 23 10 14 20  2 13  7 12 17  5  1  6  3 15  9]
[]
[40 49 23  2  8  9  1 19]


In [339]:
print(lb_sorted_index)
print(lb_sorted)
print(inactive_lower_bounds)

[19  1  9  8  2 23 49 40 25 48  5 11 22 32  4  3 12 24  6 13 20 36 27 39
 26 47 35 14 21 34 45 17 37 41  7 15 44 43 28 38 33 30 46 10 42 31 16 29
  0 18]
[-1.67197164e-03 -1.21249924e-03 -1.15577760e-03 -1.00242625e-03
 -9.28156579e-04 -6.26687751e-04 -2.57323011e-04 -8.15842025e-05
  1.80105172e-04  3.45065108e-04  3.90902511e-04  3.95549395e-04
  5.24085316e-04  6.28537406e-04  6.50858010e-04  6.68102316e-04
  6.72947009e-04  9.10559921e-04  9.67212398e-04  9.71676547e-04
  1.02293782e-03  1.07059408e-03  1.18564243e-03  1.33151144e-03
  1.34188524e-03  1.58513359e-03  1.77608703e-03  3.13687021e-03
  3.45327799e-03  3.67419217e-03  3.80371149e-03  3.84745283e-03
  8.13953554e-03  8.48031271e-03  9.59842830e-03  1.38741319e-02
  2.03259958e-02  2.05251930e-02  9.86451954e-02  9.95401493e-02
  1.00780954e-01  1.08963961e-01  1.16996085e-01  1.73567523e-01
  1.77266531e-01  2.05035916e-01  3.70764475e-01  3.90383296e-01
  6.54204116e-01  7.14406354e-01]
[]


In [340]:
print(ub_sorted_index)
print(ub_sorted)
print(inactive_upper_bounds)

[18  0 29 16 31 42 10 46 30 33 38 28 43 44 15  7 41 37 17 45 34 21 14 35
 47 26 39 27 36 20 13  6 24 12  3  4 32 22 11  5 48 25 40 49 23  2  8  9
  1 19]
[0.28559365 0.34579588 0.6096167  0.62923553 0.79496408 0.82273347
 0.82643248 0.88300392 0.89103604 0.89921905 0.90045985 0.9013548
 0.97947481 0.979674   0.98612587 0.99040157 0.99151969 0.99186046
 0.99615255 0.99619629 0.99632581 0.99654672 0.99686313 0.99822391
 0.99841487 0.99865811 0.99866849 0.99881436 0.99892941 0.99897706
 0.99902832 0.99903279 0.99908944 0.99932705 0.9993319  0.99934914
 0.99937146 0.99947591 0.99960445 0.9996091  0.99965493 0.99981989
 1.00008158 1.00025732 1.00062669 1.00092816 1.00100243 1.00115578
 1.0012125  1.00167197]
[40 49 23  2  8  9  1 19]


In [341]:
print(set(inactive_ineq_index).intersection(set(active_ineq_target)))
print(set(inactive_lower_bounds).intersection(set(active_lower_bounds)))
print(set(inactive_upper_bounds).intersection(set(active_upper_bounds)))

set()
set()
set()


In [342]:
remaining_ineq_index = np.setdiff1d(np.arange(num_ineq), inactive_ineq_index)
G_remaining = G[remaining_ineq_index]
h_remaining = h[remaining_ineq_index]

lb_remaining = Lb_np.copy().astype(np.float64)
ub_remaining = Ub_np.copy().astype(np.float64)
lb_remaining[inactive_lower_bounds] = -np.inf
ub_remaining[inactive_upper_bounds] = np.inf

In [343]:
model_reduced = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
model_reduced.setParam('OutputFlag', 1)
model_reduced.setParam('Presolve', 1)
# initialize the primal variables in model_hat2
# for i, v in enumerate(model_reduced.getVars()):
#     v.setAttr('Start', Ystar[i])
model_reduced.optimize()
optimal_val = model_reduced.objVal
time_reduced = model_reduced.Runtime + inference_time
optimal_x = np.array([v.x for v in model_reduced.getVars()])
print("Value: full={}, reduced={}, diff={}".format(optimal_val_full, optimal_val, optimal_val_full - optimal_val))
print("Time: full={}, reduced={}, diff={}".format(time_full, time_reduced, time_full - time_reduced))

Set parameter OutputFlag to value 1
Set parameter Presolve to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 14 rows, 50 columns and 387 nonzeros
Model fingerprint: 0x738980a2
Model has 1213 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-03, 4e+00]
  Objective range  [2e-01, 4e+00]
  QObjective range [2e-04, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 2355 rows, 1264 columns, 6281 nonzeros
Presolved model has 36 quadratic constraint(s)
Presolved model has 1177 bilinear constraint(s)
Variable types: 1264 continuous, 0 integer (0 binary)
Found heuristic solution: objective 4.2930270

Root relaxation: objective -2.541846e+00, 

In [331]:
full_model.Runtime, model_reduced.Runtime

(16.868891954421997, 8.8787260055542)

In [332]:
print(np.max(A_np @ optimal_x - X_np[idx]))
print(np.max(G_np @ optimal_x - h_np))

4.440892098500626e-16
8.881784197001252e-16


# osqp warm start

In [89]:
num_trial = 10

total_time = 0.0
for _ in range(num_trial):
    solver = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    # solver1.warm_start(x=Ystar)
    start_time = time.time()
    results = solver.solve()
    end_time = time.time()
    total_time += (end_time - start_time)
print(total_time / num_trial)
print(results.info.obj_val)


ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


ValueError: Workspace allocation error!

In [30]:
total_time_ws = 0.0
for _ in range(num_trial):
    solver_ws = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    solver_ws.warm_start(x=Ystar)
    start_time = time.time()
    result_ws = solver_ws.solve()
    end_time = time.time()
    total_time_ws += (end_time - start_time)
print(total_time_ws / num_trial + inference_time)
print(result_ws.info.obj_val)

0.016102910041809082
13375.848118813015


In [31]:
total_time_reduced = 0.0
for _ in range(num_trial):
    solver_reduced = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
    start_time = time.time()
    results_reduced = solver_reduced.solve()
    end_time = time.time()
    total_time_reduced = end_time - start_time
print(total_time_reduced / num_trial + inference_time)
print(results_reduced.info.obj_val)

0.0038568258285522463
13375.833319200805


In [32]:
total_time_reduced_ws = 0.0
for _ in range(num_trial):
    solver_reduced_ws = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
    solver_reduced_ws.warm_start(x=Ystar)
    start_time = time.time()
    results_reduced_ws = solver_reduced_ws.solve()
    end_time = time.time()
    total_time_reduced_ws = end_time - start_time
print(total_time_reduced_ws / num_trial + inference_time)
print(results_reduced_ws.info.obj_val)

0.003991389274597168
13375.773211036943


In [33]:
x = results_reduced.x
data.ineq_dist(X_np[idx], torch.tensor(x).unsqueeze(0)).max()

tensor(5.0846e-09)

# check

In [23]:
error = []
for i in range(9900, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    full_model.optimize()
    optimal_x_full = np.array([v.x for v in full_model.getVars()])
    active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
    active_ineq_target.tolist()

    Yhat = model(X[idx].unsqueeze(0))
    Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
    Ystar = Ystar.detach().numpy().flatten()
    ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
    ineq_sorted = np.sort(-G_np@Ystar + h_np)
    inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]

    error.append(len(set(inactive_ineq_index).intersection(set(active_ineq_target))))
print(max(error))



0


In [91]:
success = 0
for i in range(9900, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    full_model.optimize()
    if full_model.status == GRB.OPTIMAL:
        success += 1
print(success)

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 14 rows, 20 columns and 49 nonzeros
Model fingerprint: 0xbf171bbe
Model has 64 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-02, 4e+00]
  Objective range  [3e-01, 4e+00]
  QObjective range [1e-03, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 6e+00]
Presolve removed 5 rows and 3 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolved: 112 rows, 79 columns, 329 nonzeros
Presolved model has 3 quadratic constraint(s)
Presolved model has 58 bilinear constraint(s)
Variable types: 79 continuous, 0 integer (0 binary)
Found heuristi